In [ ]:
!pip install -q -U datasets peft trl accelerate bitsandbytes transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 25.3 MB/s eta 0:00:00


In [ ]:
medical_base_id = "ae-aydin/LLama-3-8B-Instruct-Medical-BNB-Base"
adapter_id = "ae-aydin/Llama-3-8B-Instruct-Medical-QLoRA"

In [ ]:
from datasets import load_dataset
import pandas as pd
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, LlamaForCausalLM
from huggingface_hub import notebook_login
from peft import PeftModel
import torch

import warnings
warnings.filterwarnings('ignore')

### Samples to Compare

In [ ]:
mm_flashcards = load_dataset("medalpaca/medical_meadow_medical_flashcards", split='train')
medquad = load_dataset("lavita/MedQuAD", split='train')

Generating train split:   0%|          | 0/33955 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/47441 [00:00<?, ? examples/s]

In [ ]:
def sample(hf, input_col, output_col):
    df = hf.to_pandas()
    df = df.dropna()
    df = df.drop_duplicates()
    df = df[df[input_col].str.len() != 0]
    df = df[df[output_col].str.len() != 0]
    df = df.sample(3, random_state=1)
    messages = list()
    outputs = list()
    for index, row in df.iterrows():
        message_dict_list = list()
        message_dict_list.append({"role": "system", "content": "Answer this question truthfully, you are a medical assistant."})
        message_dict_list.append({"role": "user", "content": str(row[input_col])})
        messages.append(message_dict_list)
        outputs.append(str(row[output_col]))
    return messages, outputs

mm_flashcards_messages, mm_flashcards_outputs = sample(mm_flashcards, "input", "output")
medquad_messages, medquad_outputs = sample(medquad, "question", "answer")

messages = mm_flashcards_messages + medquad_messages
outputs = mm_flashcards_outputs + medquad_outputs

In [ ]:
gui_examples = [
    "What are the common symptoms of influenza and how can I differentiate it from a common cold?",
    "Can you explain the potential side effects of taking amoxicillin and any precautions I should be aware of?",
    "What are some effective lifestyle changes I can make to manage high blood pressure?",
    "A woman with coronary artery disease is starting to go for a walk. As she begins, her heart rate accelerates from a resting pulse of 60 bpm until it reaches a rate of 120 bpm, at which point she begins to feel a tightening in her chest. She stops walking to rest and the tightening resolves. This has been happening to her consistently for the last 6 months. Which of the following is a true statement?? {'A': 'Increasing the heart rate increases the amount of time spent during each cardiac cycle', 'B': 'Increasing the heart rate decreases the relative amount of time spent during diastole', 'C': 'Perfusion of the myocardium takes place primarily during systole', 'D': 'Perfusion of the myocardium takes place equally throughout the cardiac cycle', 'E': 'This patient's chest pain is indicative of transmural ischemia'}",
    "A 24-year-old woman recently noticed a mass in her left breast. The examination shows a 4-cm mass in the left upper quadrant. The mass is firm, mobile, and has well-defined margins. She complains of occasional tenderness. There is no lymphatic involvement. Mammography showed a dense lesion. What is the most likely cause?? {'A': 'Ductal carcinoma in situ (DCIS)', 'B': 'Fibroadenoma', 'C': 'Phyllodes tumor', 'D': 'Inflammatory carcinoma', 'E': 'Invasive ductal carcinoma (IDC)'}"
]

def prep_examples(example_list):
    examples = list()
    for example in example_list:
        message_dict_list = list()
        message_dict_list.append({"role": "system", "content": "Answer this question truthfully, you are a medical assistant."})
        message_dict_list.append({"role": "user", "content": example})
        examples.append(message_dict_list)
    return examples

examples = prep_examples(gui_examples)

In [ ]:
def generate(model, tokenizer, messages):
    model_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map="auto",
    )

    for message in messages:
        prompt = model_pipeline.tokenizer.apply_chat_template(
                message,
                tokenize=False,
                add_generation_prompt=True
        )

        terminators = [
            model_pipeline.tokenizer.eos_token_id,
            model_pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]

        preds = model_pipeline(
            prompt,
            max_new_tokens=256,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.1,
            top_p=0.9
        )

        print()
        print("Question:")
        print(message[1]["content"])
        print("Model Generated:")
        print(preds[0]["generated_text"][len(prompt):])
        print()

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(medical_base_id, low_cpu_mem_usage=True, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(medical_base_id)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors.index.json:   0%|          | 0.00/132k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Base Quantized Model

In [ ]:
generate(base_model, tokenizer, messages)


Question:
What are some medications that can be used to treat HBV?
Model Generated:
As a medical assistant, I'm happy to provide you with information on medications used to treat Hepatitis B (HBV). Keep in mind that treatment options may vary depending on the stage and severity of the infection, as well as the individual's overall health.

Currently, there are no medications that can cure HBV, but antiviral medications can help manage the infection and reduce the risk of complications. Here are some common medications used to treat HBV:

1. Nucleoside analogues (NAs):
	* Lamivudine (Epivir): Effective against HBV, but may not completely suppress viral replication.
	* Adefovir (Hepsera): Used to treat chronic HBV, especially in patients with liver cirrhosis.
	* Entecavir (Baraclude): A potent NA that can suppress HBV replication and reduce liver inflammation.
	* Telbivudine (Tyzeka): Another NA that can reduce HBV replication and liver inflammation.
2. Pegylated interferons (Peg-IFNs):

In [ ]:
generate(base_model, tokenizer, examples)


Question:
What are the common symptoms of influenza and how can I differentiate it from a common cold?
Model Generated:
As a medical assistant, I'd be happy to help you identify the common symptoms of influenza (the flu) and differentiate it from a common cold.

**Common Symptoms of Influenza (the Flu):**

1. **Fever**: A high temperature, usually above 102°F (39°C), is a hallmark symptom of the flu.
2. **Chills**: Many people experience chills or a feeling of coldness, even if their body temperature is normal.
3. **Cough**: A dry, hacking cough is common, but it can also be accompanied by a productive cough.
4. **Sore Throat**: A scratchy, sore throat is a common symptom of the flu.
5. **Headache**: A headache, often described as a dull, throbbing pain, is a common complaint.
6. **Fatigue**: Feeling extremely tired, weak, or exhausted is a common symptom of the flu.
7. **Muscle Pain**: Sore, achy muscles, especially in the back, arms, and legs, are common.
8. **Runny or Stuffy Nose**

## LoRA Fine-tuned Model

In [ ]:
finetuned_model = PeftModel.from_pretrained(base_model, adapter_id)

adapter_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

In [ ]:
generate(finetuned_model, tokenizer, messages)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL


Question:
What are some medications that can be used to treat HBV?
Model Generated:
Some medications that can be used to treat HBV include lamivudine, adefovir, tenofovir, and entecavir. These medications are known as antiviral agents and work by inhibiting the replication of the HBV virus in the body. They can be used to treat chronic HBV infection, as well as to prevent the spread of the virus from mother to child during pregnancy. It is important to note that the choice of medication will depend on the individual patient's needs and medical history, and that treatment should be monitored by a healthcare provider. Additionally, some patients may require combination therapy with multiple medications to achieve optimal results. It is important to follow the recommended treatment regimen and to report any side effects or concerns to a healthcare provider. With proper treatment, many patients with HBV can achieve a sustained response and reduce the risk of complications. However, it is 

In [ ]:
generate(finetuned_model, tokenizer, examples)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL


Question:
What are the common symptoms of influenza and how can I differentiate it from a common cold?
Model Generated:
Influenza is a viral infection that can cause a range of symptoms, including fever, cough, sore throat, and body aches. While a common cold can also cause these symptoms, influenza is typically more severe and can lead to complications such as pneumonia. Influenza is also more contagious than a common cold, and can spread quickly through communities. It is important to seek medical attention if you are experiencing symptoms of influenza, as antiviral medications can help to reduce the severity and duration of the illness. Additionally, getting vaccinated against influenza can help to prevent the spread of the virus and reduce the risk of complications. If you are unsure whether you have a common cold or influenza, it is important to consult with a healthcare professional for an accurate diagnosis and appropriate treatment. They can perform a physical examination, tak

In [ ]:
print(*outputs, sep='\n\n')

Entecavir, tenofovir, and PEG-IFN-alpha can be used to treat HBV.

Candida vaginitis (Candida albicans infection) is the cause of vaginitis that is characterized by a thick, white discharge and vaginal inflammation.

A major side effect of ribavirin is that it may cause dose-dependent hemolytic anemia in 10-20% of patients.

Leishmaniasis is a parasitic disease spread by the bite of infected sand flies. There are several different forms of leishmaniasis. The most common are cutaneous and visceral. The cutaneous type causes skin sores. The visceral type affects internal organs such as the spleen, liver, and bone marrow. People with this form usually have fever, weight loss, and an enlarged spleen and liver.     Leishmaniasis is found in parts of about 88 countries. Most of these countries are in the tropics and subtropics. It is possible but very unlikely that you would get this disease in the United States. But you should be aware of it if you are traveling to the Middle East or parts 